In [1]:
import gurobipy as gp
import numpy as np

model = gp.Model()
model.setParam('Method', 0)

x_j = model.addMVar(2, lb=0)

# Objective
model.setObjective(x_j[0] + x_j[1], gp.GRB.MAXIMIZE)

# Constraints
model.addConstr(2 * x_j[0] + x_j[1] <= 2, "c0")
model.addConstr(x_j[0] + 2 * x_j[1] <= 2, "c1")

model.update()
# constraints = model.getConstrs()

# constraints[0].CBasis = - 1
# constraints[1].CBasis = - 1

# # x_j.VBasis = [0, 0]
# variables = model.getVars()
# variables[0].VBasis = 0
# variables[1].VBasis = 0


# model.optimize()

Set parameter TSPort to value 61954
Set parameter TokenServer to value "10.0.1.1"
Set parameter Method to value 0


# Using .sol

In [2]:
import gurobipy as gp
import numpy as np

model = gp.Model()
model.setParam('Method', 0)

x_j = model.addMVar(2, lb=0)

# Objective
model.setObjective(x_j[0] + x_j[1], gp.GRB.MAXIMIZE)

# Constraints
model.addConstr(2 * x_j[0] + x_j[1] <= 2, "c0")
model.addConstr(x_j[0] + 2 * x_j[1] <= 2, "c1")

model.optimize()


model.write('test.sol')

# Save model
model.write('test.mps')
model.write('test.bas')

print(model.x)

Set parameter Method to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
Method  0
TSPort  61954

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xf17fe891
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   2.000000e+00      0s
       2    1.3333333e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.333333333e+00
[0.6666666666666666, 0.6666666666666667]


In [3]:
model.addConstr(x_j[0] + 5* x_j[1] <= 2, "c3")
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
Method  0
TSPort  61954

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3333333e+00   2.500000e-01   7.499993e+05      0s
       1    1.1111111e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.111111111e+00


In [4]:
model = gp.read('test.mps')
x_j = model.getVars()
model.addConstr(x_j[0] + 5* x_j[1] <= 2, "c3")
model.update()
model.read('test.bas')
# model.optimize()
# print(model.x)

Read MPS format model from file test.mps
Reading time = 0.00 seconds
: 2 rows, 2 columns, 4 nonzeros
Read basis from file test.bas


In [5]:
print(model.getVars()[0].VBasis)
print(model.getConstrs()[2].CBasis)

0
0


# Master pb

In [108]:
path = 'GMM_quad_v1-50/output/'

master_pb = gp.read(path + 'master_pb.mps')
master_pb.setParam('Method', 0)

lambda_k = gp.tupledict({k: master_pb.getVarByName(f"parameters[{k}]") for k in range(5)})
master_pb.addConstr(lambda_k[0] + lambda_k[1] <= 1)
master_pb.update()

master_pb.read(path +'master_pb.bas')
# master_pb.update()

Read MPS format model from file GMM_quad_v1-50/output/master_pb.mps
Reading time = 0.02 seconds
GMM_pb: 12752 rows, 13248 columns, 49902 nonzeros
Set parameter Method to value 0


Read basis from file GMM_quad_v1-50/output/master_pb.bas


In [109]:
master_pb.optimize()
print(master_pb.x[:5])

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
Method  0
TSPort  61954

Optimize a model with 12753 rows, 13248 columns and 49904 nonzeros
Coefficient statistics:
  Matrix range     [1e-05, 5e+03]
  Objective range  [2e-02, 6e+03]
  Bounds range     [1e+09, 1e+09]
  RHS range        [2e-03, 2e+01]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.4353834e+00   3.166705e+02   1.129660e+08      0s
     647   -1.0211789e+01   0.000000e+00   0.000000e+00      0s

Solved in 647 iterations and 0.06 seconds (0.07 work units)
Optimal objective -1.021178911e+01
[0.11893582344055176, 0.8810641765594482, 8.606916785240173, 3827.849734067917, 0.0]


In [41]:
basic = [var for var in master_pb.getVars() if var.VBasis == 0]
non_basic = [var for var in master_pb.getVars() if var.VBasis == -1]

In [42]:
len(basic) 

12752